In [0]:
!pip install -q Sastrawi

    100% |████████████████████████████████| 215kB 20.2MB/s 


In [0]:
!pip install -q python-crfsuite

    100% |████████████████████████████████| 706kB 17.2MB/s 


In [0]:
!pip install -q nltk

In [0]:
!pip install -q textblob

In [0]:
#import library
import pandas as pd
import numpy as np
import nltk
import Sastrawi
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from nltk.tag import CRFTagger
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('punkt')
import pycrfsuite
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from textblob import TextBlob


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [0]:
# ngikutin tutorial
# butuh all_indo_man_tag_corpus_model, negatif_vania, negative_koto, positif vania, positif koto, stopwords
def normalisasi(tweet):
  normal_tw = tweet.lower() #lowercase
  normal_tw = re.sub('\s+', ' ', normal_tw) # remove extraspace
  normal_tw = normal_tw.strip() #trim depan belakang
  normal_tw = re.sub(r'[^\w\s]','',normal_tw) #buang punctuation, 
  normal_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE)
  #regex huruf yang berulang kaya haiiii (untuk fitur unigram)
  normal_tw = normal_regex.sub(r"\1\1", normal_tw) #buanghuruf yang berulang
  return normal_tw

def remove_stopwords(tweet):
  stopwords = pd.read_csv("stopwords.csv")
  special_list = ['username', 'url', 'sensitive-no']
  token = nltk.word_tokenize(tweet)
  token_afterremoval = []
  for k in token:
    if k not in stopwords and k not in special_list:
      token_afterremoval.append(k)

  str_clean = ' '.join(token_afterremoval)
  return str_clean

def Stemming(tweet):
  token = nltk.word_tokenize(tweet)
  stem_kalimat = []
  for k in token:
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stem_kata = stemmer.stem(k.encode('ascii','ignore'))
    stem_kalimat.append(stem_kata)

  stem_kalimat_str = ' '.join(stem_kalimat)
  return stem_kalimat_str


def pra_pemrosesan(list_tweet):
  tweet_clean = []
  for tw in list_tweet:
    normal_tweet = normalisasi(tw)
    nosw_tweet = remove_stopwords(normal_tweet)
#     stem_tweet = Stemming(nosw_tweet)
    tweet_clean.append(nosw_tweet)
  return tweet_clean

def EkstraksiBoW(tweet):
  unigram = CountVectorizer(ngram_range=(1,1), max_features=2000)
  unigram_matrix = unigram.fit_transform(np.array(tweet)).todense()
  nama_fitur = unigram.get_feature_names()
  return unigram_matrix, nama_fitur


def EkstraksiSentimen(list_tweet):
  pos = pd.read_csv("positif_vania.txt", header=None, names=['pos'])
  list_pos = pos['pos'].tolist()
  neg = pd.read_csv("negatif_vania.txt", header=None, names=['neg'])
  list_neg = neg['neg'].tolist()
  
  fitur_sentimen_all = []
  for tweet in list_tweet:
    ##inisiasi value
    emosi = ["positif", "negatif"]
    value = [0,0]
    emosi_value = {}
    for i in range(len(emosi)):
      emosi_value[emosi[i]] = value[i]
    
    list_kata = tweet.split()
  
    for k in list_kata:
      if k in list_pos:
        emosi_value["positif"] += 1
      if k in list_neg:
        emosi_value["negatif"] += 1
    fitur_sentimen_perkalimat = list(emosi_value.values())
    fitur_sentimen_all.append(fitur_sentimen_perkalimat)
  return fitur_sentimen_all


def EkstraksiPOS(list_tweet):
  ct = CRFTagger()
  
  ct.set_model_file("all_indo_man_tag_corpus_model.crf.tagger")
  pos_feat_list = []
  count_tag = []
  for tweet in list_tweet:
    token = nltk.word_tokenize(tweet)
    tag = ct.tag_sents([token])
    flat_tag = [item for sublist in tag for item in sublist]
    pos_count = Counter([j for i,j in flat_tag])
    pos_feat = (pos_count['JJ'], pos_count['NEG'])
    pos_feat_list.append(pos_feat)
  return pos_feat_list


def EkstraksiOrtografi(raw_tweet):
  all_orto_feat = []
  for tw in raw_tweet:
    capital_count = sum(1 for c in tw if c.isupper())
    exclamation_count = sum((1 for c in tw if c == "!"))
    word_len = len(nltk.word_tokenize(tw))
    char_len = len(tw)
    orto_feat = [capital_count, exclamation_count, word_len, char_len]
    all_orto_feat.append(orto_feat)
  return all_orto_feat

def EkstraksiSentimenKoto(list_tweet):
  pos = pd.read_csv("positive_koto.tsv", delimiter="\t", index_col=False, header=None, names=['pos', 'sentiment'])
  list_pos = pos['pos'].tolist()
  list_pos_sent = pos['sentiment'].tolist()
  neg = pd.read_csv("negative_koto.tsv", delimiter="\t", index_col=False, header=None, names=['neg', 'sentiment'])
  list_neg = neg['neg'].tolist()
  list_neg_sent = neg['sentiment'].tolist()
  
  dict_pos = dict(zip(list_pos, list_pos_sent))
  dict_neg = dict(zip(list_neg, list_neg_sent))
  
  fitur_sentimen_all = []
  for tweet in list_tweet:
    ##inisiasi value
    emosi = ["positif", "negatif"]
    value = [0,0]
    emosi_value = {}
    for i in range(len(emosi)):
      emosi_value[emosi[i]] = value[i]
    list_kata = tweet.split()
    list_key_pos = dict_pos.keys()
    list_key_neg = dict_neg.keys()
    for k in list_kata:
      if k in list_key_pos:
        emosi_value["positif"] += dict_pos.get(k)
      if k in list_key_neg:
        emosi_value["negatif"] += -(dict_neg.get(k))
    fitur_sentimen_perkalimat = list(emosi_value.values())
    fitur_sentimen_all.append(fitur_sentimen_perkalimat)
  return fitur_sentimen_all



  
  


In [0]:
#membaca data
raw_data = pd.read_csv("train_set.csv",
delimiter=",", encoding="Latin-1")
raw_data.head()

,id,sentimen,tweet
0,1,1,oks kak semangat ya kalian kalian
1,2,0,sekarang harus kaya orang bodoh lagi bodoh sangat
2,3,1,"Begitu diumumkan lulus 100%, mereka semua suju..."
3,4,0,[USERNAME] [USERNAME] Katanya Bapak Reformasi ...
4,5,0,macet macetan perut kosong akhirnya mampir dah...


In [0]:
raw_tweet = raw_data['tweet']
label = raw_data['sentimen'].tolist()
clean_tweet = pra_pemrosesan(raw_tweet)
clean_tweet[:3]

[u'oks kak semangat ya kalian kalian',
 u'sekarang harus kaya orang bodoh lagi bodoh sangat',
 u'begitu diumumkan lulus 100 mereka semua sujud syukur dan langsung mengambil bungasaat dia menghampiri langsung memeluk menciumku air mata tak kuasa kubendungmom this is my birthday present for u']

In [0]:
clean_tweet[:3]

[u'oks kak semangat ya kalian kalian',
 u'sekarang harus kaya orang bodoh lagi bodoh sangat',
 u'begitu diumumkan lulus 100 mereka semua sujud syukur dan langsung mengambil bungasaat dia menghampiri langsung memeluk menciumku air mata tak kuasa kubendungmom this is my birthday present for u']

In [0]:
unigram_feat, feat_name = EkstraksiBoW(clean_tweet)
print(unigram_feat[:3])
print(feat_name[:10])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
[u'10', u'100', u'11', u'12', u'20', u'2017', u'2018', u'24', u'30', u'aa']


In [0]:
sentlex_feat = EkstraksiSentimen(clean_tweet)
print(sentlex_feat[:10])

[[1, 0], [0, 3], [1, 0], [2, 0], [0, 2], [0, 2], [0, 0], [2, 0], [1, 0], [0, 0]]


In [0]:
postag_feat = EkstraksiPOS(clean_tweet)
print(postag_feat[:3])

[(0, 0), (1, 0), (0, 1)]


In [0]:
orto_feat = EkstraksiOrtografi(raw_tweet)
orto_feat[:3]

[[0, 0, 6, 33], [0, 0, 8, 49], [4, 0, 46, 219]]

In [0]:
sentlex_koto_feat = EkstraksiSentimenKoto(clean_tweet)
print(sentlex_koto_feat[:10])

[[8, 1], [5, 15], [27, 10], [5, 42], [7, 8], [13, 20], [11, 13], [19, 14], [32, 19], [5, 21]]


In [0]:
feat_list = [unigram_feat, sentlex_feat, postag_feat, orto_feat, sentlex_koto_feat]
feat_name = ["Unigram", "Sentimen", "POS", "Ortografi", "Sentimen Koto"]


In [0]:
for f, n in zip(feat_list, feat_name):
  X = f
  y = label
  scoring = ['f1_macro']
  print("LOGISTIC REGRESSION")
  lg = LogisticRegression()
  scores = cross_validate(lg, X, y, cv=10, scoring=scoring)
  f1 = np.mean(scores['test_f1_macro'])
  print("Jenis Fitur : ", n)
  print("F1-Score :", f1)
  print("---------------")
  print("DECISION TREE")
  dtc = DecisionTreeClassifier()
  scores = cross_validate(dtc, X, y, cv=10, scoring=scoring)
  f1 = np.mean(scores['test_f1_macro'])
  print("Jenis Fitur : ", n)
  print("F1-Score :", f1)
  print("---------------")
  

LOGISTIC REGRESSION
('Jenis Fitur : ', 'Unigram')
('F1-Score :', 0.8476446145265975)
---------------
DECISION TREE
('Jenis Fitur : ', 'Unigram')
('F1-Score :', 0.7676955687933508)
---------------
LOGISTIC REGRESSION
('Jenis Fitur : ', 'Sentimen')
('F1-Score :', 0.7916838705373135)
---------------
DECISION TREE
('Jenis Fitur : ', 'Sentimen')
('F1-Score :', 0.7890882123922349)
---------------
LOGISTIC REGRESSION
('Jenis Fitur : ', 'POS')
('F1-Score :', 0.5170408403335769)
---------------
DECISION TREE
('Jenis Fitur : ', 'POS')
('F1-Score :', 0.5065480145412494)
---------------
LOGISTIC REGRESSION
('Jenis Fitur : ', 'Ortografi')
('F1-Score :', 0.5260106021289268)
---------------
DECISION TREE
('Jenis Fitur : ', 'Ortografi')
('F1-Score :', 0.5251676673279936)
---------------
LOGISTIC REGRESSION
('Jenis Fitur : ', 'Sentimen Koto')
('F1-Score :', 0.6885957118208752)
---------------
DECISION TREE
('Jenis Fitur : ', 'Sentimen Koto')
('F1-Score :', 0.6382555178482971)
---------------


In [0]:
# untuk test set

#membaca data
raw_data_tester = pd.read_csv("test_set.csv",
delimiter=",", encoding="Latin-1")
raw_data_tester.head()



,test_ID,tweet
0,0,Jadi wanita jangan suka menghancurkan hubungan...
1,1,sombong apanya kalau sms saja dibls terus
2,2,apadah kamu :p cie cie baik kamu cie bebe cie ...
3,3,tdrlah besok medical check up semoga lancar â?...
4,4,crew serbu bsm seru (at bank syariah mandiri b...


In [0]:
raw_tweet_tester = raw_data_tester['tweet']
clean_tweet_tester = pra_pemrosesan(raw_tweet_tester)
clean_tweet_tester[:3]

[u'jadi wanita jangan suka menghancurkan hubungan orang jgn bangga berhasil merusak kebahagian orang silahkan saja tapi ga berkah bahagianya nanti hehe',
 u'sombong apanya kalau sms saja dibls terus',
 u'apadah kamu p cie cie baik kamu cie bebe cie kiwkiw']

In [0]:
unigram_feat_tester, feat_name_tester = EkstraksiBoW(clean_tweet_tester)
print('unigram')
print(unigram_feat_tester[:3])
print(feat_name_tester)

print('sentlex')
sentlex_feat_tester = EkstraksiSentimen(clean_tweet_tester)
print(sentlex_feat_tester[:10])

print('postag')
postag_feat_tester = EkstraksiPOS(clean_tweet_tester)
print(postag_feat_tester[:3])

print('orto')
orto_feat_tester = EkstraksiOrtografi(raw_tweet_tester)
orto_feat_tester[:3]

print('sentlex koto')
sentlex_koto_feat_tester = EkstraksiSentimenKoto(clean_tweet_tester)
print(sentlex_koto_feat_tester[:10])


unigram
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[u'10', u'12', u'20', u'2018', u'2019tetapjokowi', u'aa', u'abang', u'abis', u'absen', u'acara', u'ad', u'adalah', u'adanya', u'ade', u'adek', u'adem', u'adik', u'adil', u'admin', u'aduh', u'ae', u'agak', u'agama', u'agar', u'agung', u'ah', u'ahaha', u'ahahah', u'ahahaha', u'ahok', u'aib', u'aih', u'aing', u'air', u'aja', u'ajak', u'ajar', u'ajarin', u'ak', u'akan', u'akhir', u'akhirnya', u'akibat', u'aku', u'akun', u'al', u'alah', u'alam', u'alasan', u'alay', u'alhamdulilah', u'alhamdulillah', u'all', u'allah', u'always', u'am', u'ama', u'aman', u'amat', u'ambil', u'amien', u'amin', u'amp', u'ampe', u'ampun', u'an', u'anak', u'anak2', u'anakanak', u'anaknya', u'and', u'anda', u'android', u'aneh', u'angap', u'anggota', u'angin', u'angka', u'anjir', u'antar', u'antara', u'anti', u'antri', u'antrian', u'ap', u'apa', u'apa2', u'apakah', u'apalagi', u'apan', u'apanya', u'apapun', u'apasih', u'ape', u'api', u'arab', u'arah', 

In [0]:
feat_list_tester = [unigram_feat_tester, sentlex_feat_tester, postag_feat_tester, orto_feat_tester, sentlex_koto_feat_tester]
feat_name_tester = ["Unigram", "Sentimen", "POS", "Ortografi", "Sentimen Koto"]



In [0]:
x_sentlex = np.array(sentlex_feat)
x_sentlex_test = np.array(sentlex_feat_tester)

x_sentlex

array([[1, 0],
       [0, 3],
       [1, 0],
       ...,
       [1, 1],
       [0, 2],
       [0, 1]])

In [0]:
lg_tester = LogisticRegression()
lg_tester.fit(x_sentlex, label)

pred_sentlex = lg_tester.predict(x_sentlex_test)
pred_sentlex

array([1, 0, 1, ..., 0, 1, 0])

In [0]:
id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_sentlex})
df_out.to_csv('submission_sentlex.csv', index=False, header=False) # 


In [0]:
submission = pd.read_csv("submission_sentlex.csv",
delimiter=",", encoding="Latin-1")

submission # hasilnya 79.52

,0,1
0,1,0
1,2,1
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,1
8,9,1
9,10,1


In [0]:
postag_sentlex = [sentlex_feat, postag_feat]
postag_sentlex_tester = [sentlex_feat_tester, postag_feat_tester]

x_postag_sentlex = np.array(postag_sentlex)
x_postag_sentlex_tester = np.array(postag_sentlex_tester)

In [0]:
lr= LogisticRegression()
lr.fit(postag_feat, label)

pred_postag_sentlex = lr.predict(postag_feat_tester)
pred_postag_sentlex

array([1, 1, 1, ..., 1, 1, 0])

In [0]:
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_postag_sentlex})
df_out.to_csv('submission_postag.csv', index=False, header=False)  #  hasilnya 52.12


In [0]:
lr_orto = LogisticRegression()
lr_orto.fit(orto_feat, label)

pred_orto = lr_orto.predict(orto_feat_tester)
pred_orto

array([1, 1, 0, ..., 0, 0, 0])

In [0]:
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_orto})
df_out.to_csv('submission_orto.csv', index=False, header=False)  # hasilnya 48.25


In [0]:
# buat gabung2 fitur
sentlex_vania_koto = map(list.__add__, sentlex_feat, sentlex_koto_feat)
sentlex_vania_koto_tester = map(list.__add__, sentlex_feat_tester, sentlex_koto_feat_tester)
sentlex_vania_koto

[[1, 0, 8, 1],
 [0, 3, 5, 15],
 [1, 0, 27, 10],
 [2, 0, 5, 42],
 [0, 2, 7, 8],
 [0, 2, 13, 20],
 [0, 0, 11, 13],
 [2, 0, 19, 14],
 [1, 0, 32, 19],
 [0, 0, 5, 21],
 [0, 2, 4, 16],
 [0, 1, 11, 14],
 [1, 0, 0, 9],
 [2, 0, 15, 20],
 [1, 0, 2, 4],
 [0, 1, 1, 6],
 [1, 1, 26, 27],
 [4, 0, 32, 23],
 [0, 1, 6, 4],
 [3, 0, 22, 38],
 [1, 0, 15, 22],
 [2, 0, 6, 12],
 [0, 0, 1, 18],
 [0, 0, 1, 16],
 [0, 1, 3, 11],
 [1, 0, 16, 12],
 [0, 1, 1, 5],
 [0, 0, 19, 30],
 [0, 1, 14, 18],
 [0, 0, 9, 19],
 [0, 0, 14, 27],
 [2, 0, 16, 7],
 [0, 2, 5, 20],
 [1, 0, 16, 35],
 [1, 1, 15, 30],
 [0, 0, 8, 3],
 [0, 0, 19, 35],
 [1, 0, 4, 6],
 [0, 1, 0, 8],
 [1, 1, 21, 23],
 [1, 0, 16, 13],
 [4, 0, 28, 49],
 [1, 2, 17, 20],
 [1, 0, 12, 10],
 [0, 0, 7, 3],
 [0, 2, 3, 23],
 [1, 1, 17, 27],
 [0, 1, 3, 12],
 [1, 0, 11, 3],
 [1, 0, 17, 16],
 [0, 0, 11, 20],
 [3, 0, 10, 15],
 [1, 0, 0, 0],
 [1, 0, 21, 17],
 [0, 0, 7, 12],
 [1, 0, 2, 5],
 [0, 2, 4, 11],
 [0, 1, 4, 12],
 [5, 1, 20, 44],
 [1, 0, 9, 12],
 [3, 0, 15, 25],
 [1, 0,

In [0]:
lr_vania_koto = LogisticRegression()
lr_vania_koto.fit(sentlex_vania_koto, label)

pred_vania_koto = lr_vania_koto.predict(sentlex_vania_koto_tester)
pred_vania_koto

array([1, 0, 1, ..., 0, 1, 0])

In [0]:
id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_vania_koto})
df_out.to_csv('submission_vania_koto.csv', index=False, header=False) # 

submission = pd.read_csv("submission_vania_koto.csv",
delimiter=",", encoding="Latin-1")

#hasilnya 80.80

In [0]:
#unigram
cv = CountVectorizer(ngram_range=(1,1), max_features=100000)
unigram = cv.fit(np.array(raw_tweet))

matrix_train = unigram.transform(raw_tweet)
matrix_tester = unigram.transform(raw_tweet_tester)

lr_unigram = LogisticRegression()
lr_unigram.fit(matrix_train, label)

pred_unigram = lr_unigram.predict(matrix_tester)
print(pred_unigram)

id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_unigram})
df_out.to_csv('submission_unigram.csv', index=False, header=False) # 

submission = pd.read_csv("submission_unigram.csv",
delimiter=",", encoding="Latin-1")

submission[:3] # hasilnya 85.68

[0 0 1 ... 0 1 0]


,0,0.1
0,1,0
1,2,1
2,3,1


In [0]:
#tf idf

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=20000)
matrix = tfidf.fit(np.array(raw_tweet))

tfidf_train = matrix.transform(raw_tweet)
tfidf_tester = matrix.transform(raw_tweet_tester)

lr_tfidf = LogisticRegression()
lr_tfidf.fit(tfidf_train, label)

pred_tfidf = lr_tfidf.predict(tfidf_tester)
print(pred_tfidf[:3])

id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_tfidf})
df_out.to_csv('submission_tfidf.csv', index=False, header=False) # 

submission = pd.read_csv("submission_tfidf.csv",
delimiter=",", encoding="Latin-1")

submission[:3] # 86.05

[0 0 1]


,0,0.1
0,1,0
1,2,1
2,3,1


In [0]:
xg_reg = xgb.XGBClassifier()

xg_reg.fit(train,label)

pred2 = xg_reg.predict(unigram_matrix_test)
pred2 #76.46

NameError: ignored

In [0]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(train, label)

pred2 = clf.predict(unigram_matrix_test)
pred2 #82.78

In [0]:
clf=RandomForestClassifier(n_estimators=100)
clf.fit(train, label)

pred2 = clf.predict(unigram_matrix_test)
pred2 #83.37

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.27310829, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [0]:
#tfidf +vania+ koto
vania_koto_tfidf_train = map(list.__add__, sentlex_vania_koto, tfidf_train.toarray().tolist())
vania_koto_tfidf_tester = map(list.__add__, sentlex_vania_koto_tester, tfidf_tester.toarray().tolist())

lr_vania_koto_tfidf = LogisticRegression()
lr_vania_koto_tfidf.fit(vania_koto_tfidf_train, label)

pred_vania_koto_tfidf = lr_vania_koto_tfidf.predict(vania_koto_tfidf_tester)
print(pred_vania_koto_tfidf[:3])

id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_vania_koto_tfidf})
df_out.to_csv('submission_vania_koto_tfidf.csv', index=False, header=False) # 

submission = pd.read_csv("submission_vania_koto_tfidf.csv",
delimiter=",", encoding="Latin-1")

submission[:3] # 86.16 -- raw tweet

[1 0 1]


,0,1
0,1,0
1,2,1
2,3,1


In [0]:
tfidf_unigram_train = map(list.__add__, matrix_train.toarray().tolist(), tfidf_train.toarray().tolist())
tfidf_unigram_tester = map(list.__add__, matrix_tester.toarray().tolist(), tfidf_tester.toarray().tolist())

lr_tfidf_unigram = LogisticRegression()
lr_tfidf_unigram.fit(tfidf_unigram_train, label)

pred_tfidf_unigram = lr_tfidf_unigram.predict(tfidf_unigram_tester)
print(pred_tfidf_unigram[:3])

id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_tfidf_unigram})
df_out.to_csv('submission_tfidf_unigram.csv', index=False, header=False) # 

submission = pd.read_csv("submission_tfidf_unigram.csv",
delimiter=",", encoding="Latin-1")

submission[:3] # 86.05

[1 0 1]


,0,1
0,1,0
1,2,1
2,3,1


In [0]:
#tf idf clean tweet -- aneh

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=2000)
matrix = tfidf.fit(np.array(clean_tweet))

tfidf_train = matrix.transform(clean_tweet)
tfidf_tester = matrix.transform(clean_tweet_tester)

lr_tfidf = LogisticRegression()
lr_tfidf.fit(tfidf_train, label)

pred_tfidf = lr_tfidf.predict(tfidf_tester)
print(pred_tfidf[:3])

id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_tfidf})
df_out.to_csv('submission_tfidf_clean.csv', index=False, header=False) # 

submission = pd.read_csv("submission_tfidf_clean.csv",
delimiter=",", encoding="Latin-1")

submission[:3] # 85.6 

[0 0 1]


,0,0.1
0,1,0
1,2,1
2,3,1


In [0]:
#unigram custom2 teruuss wkwk
cv = CountVectorizer(ngram_range=(1,1), max_features=100000)
unigram = cv.fit(np.array(raw_tweet))

matrix_train = unigram.transform(raw_tweet)
matrix_tester = unigram.transform(raw_tweet_tester)

lr_unigram = LogisticRegression()
lr_unigram.fit(matrix_train, label)

pred_unigram = lr_unigram.predict(matrix_tester)
print(pred_unigram)

id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_unigram})
df_out.to_csv('submission_unigram_custom.csv', index=False, header=False) # 

submission = pd.read_csv("submission_unigram_custom.csv",
delimiter=",", encoding="Latin-1")

submission[:3] # hasilnya 85.68

[0 0 1 ... 0 1 0]


,0,0.1
0,1,0
1,2,1
2,3,1


In [0]:
bayes_vania_koto_tfidf = MultinomialNB()
bayes_vania_koto_tfidf.fit(vania_koto_tfidf_train, label)

pred_vania_koto_tfidf = bayes_vania_koto_tfidf.predict(vania_koto_tfidf_tester)
print(pred_vania_koto_tfidf[:3])

id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred_vania_koto_tfidf})
df_out.to_csv('submission_bayes_vania_koto_tfidf.csv', index=False, header=False) # 

submission = pd.read_csv("submission_bayes_vania_koto_tfidf.csv",
delimiter=",", encoding="Latin-1")

submission[:3] # 86.16 -- raw tweet

[1 0 1]


,0,1
0,1,0
1,2,1
2,3,1


In [0]:
from sklearn.naive_bayes import MultinomialNB

cv = CountVectorizer(ngram_range=(1,1), max_features=6000)
unigram = cv.fit(np.array(clean_tweet))

# vectorizer = TfidfVectorizer(max_features = 6000)

matrix_train = unigram.transform(clean_tweet)
matrix_tester = unigram.transform(clean_tweet_tester)

clf = MultinomialNB()
model =clf.fit(matrix_train, label)
# model = clf.fit(vectorizer.fit_transform(clean_tweet), label)


predictions = model.predict(matrix_tester)

print(predictions[:3])

id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':predictions})
df_out.to_csv('submission_multinomial_unigram.csv', index=False, header=False)

submission = pd.read_csv("submission_multinomial_unigram.csv", delimiter=",", encoding="Latin-1")
submission

[1 0 1]


,0,1
0,1,0
1,2,1
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,0
8,9,1
9,10,1


In [0]:
from sklearn.naive_bayes import MultinomialNB

# cv = CountVectorizer(ngram_range=(1,1), max_features=10000)
# unigram = cv.fit(np.array(clean_tweet))

vectorizer = TfidfVectorizer(max_features = 6000)

# matrix_train = unigram.transform(raw_tweet)
# matrix_tester = unigram.transform(raw_tweet_tester)

clf = MultinomialNB()
# clf.fit(matrix_train, label)
model = clf.fit(vectorizer.fit_transform(clean_tweet), label)


# predictions = clf.predict(clean_tweet_tester)
predictions = clf.predict(vectorizer.transform(clean_tweet_tester))
print(predictions)

id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':predictions})
df_out.to_csv('submission_multinomial_tfidf.csv', index=False, header=False)

submission = pd.read_csv("submission_multinomial_tfidf.csv", delimiter=",", encoding="Latin-1")
submission[:3]

[1 0 1 ... 0 1 0]


,0,1
0,1,0
1,2,1
2,3,1


In [0]:
from sklearn.svm import SVC

svc = SVC(gamma=0.01, max_iter = 1000000)

svc.fit(tfidf_train, label)
pred= svc.predict(tfidf_tester)

id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':pred})
df_out.to_csv('submission_svc_tfidf.csv', index=False, header=False)

submission = pd.read_csv("submission_svc_tfidf.csv", delimiter=",", encoding="Latin-1")
submission[:3]

,0,0.1
0,1,0
1,2,0
2,3,0


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm

vectorizer = TfidfVectorizer(max_features = 6600)

SVM = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')
model = SVM.fit(vectorizer.fit_transform(clean_tweet), label)

predictions = model.predict(vectorizer.transform(clean_tweet_tester))
print(predictions)

id_target = raw_data_tester['test_ID']
df_out = pd.DataFrame({'id':id_target, 'sentimen':predictions})
df_out.to_csv('submission_tfidf_svm.csv', index=False, header=False)

submission = pd.read_csv("submission_tfidf_svm.csv", delimiter=",", encoding="Latin-1")
submission

[1 0 1 ... 0 1 0]


,0,1
0,1,0
1,2,1
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,0
8,9,1
9,10,1
